## Fake News Prediction

### Importing the libraries

In [27]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [12]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#### printing the stopwords in English

In [13]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

### Loading the dataset

In [14]:
news_dataset = pd.read_csv('Datasets/News_DATA.csv')

In [15]:
news_dataset.shape

(20800, 5)

In [16]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \r\nAn Iranian woman has been sentenced ...,1


### Checking the missing values

In [17]:
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

### Replacing the null values with empty string

In [18]:
news_dataset = news_dataset.fillna('')

In [19]:
news_dataset.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

### Merging the author name and news title

In [20]:
news_dataset['content'] = news_dataset['author']+''+news_dataset['title']

In [21]:
news_dataset['content']

0        Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1        Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom...
2        Consortiumnews.comWhy the Truth Might Get You ...
3        Jessica Purkiss15 Civilians Killed In Single U...
4        Howard PortnoyIranian woman jailed for fiction...
                               ...                        
20795    Jerome HudsonRapper T.I.: Trump a ’Poster Chil...
20796    Benjamin HoffmanN.F.L. Playoffs: Schedule, Mat...
20797    Michael J. de la Merced and Rachel AbramsMacy’...
20798    Alex AnsaryNATO, Russia To Hold Parallel Exerc...
20799               David SwansonWhat Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object

### Separating the data and label

In [22]:
X = news_dataset.drop(columns='label', axis=1)
y = news_dataset['label']

In [23]:
print(X)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [24]:
print(y)

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


### Stemming
##### - to reduce a word into root word

In [25]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [28]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [29]:
news_dataset['content']

0        darrel lucushous dem aid even see comey letter...
1        daniel j flynnflynn hillari clinton big woman ...
2                consortiumnew comwhi truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoyiranian woman jail fiction unpub...
                               ...                        
20795    jerom hudsonrapp trump poster child white supr...
20796    benjamin hoffmann f l playoff schedul matchup ...
20797    michael j de la merc rachel abramsmaci said re...
20798    alex ansarynato russia hold parallel exercis b...
20799                        david swansonwhat keep f aliv
Name: content, Length: 20800, dtype: object

### Separating the data and target

In [30]:
X = news_dataset['content'].values
y = news_dataset['label'].values

In [31]:
X

array(['darrel lucushous dem aid even see comey letter jason chaffetz tweet',
       'daniel j flynnflynn hillari clinton big woman campu breitbart',
       'consortiumnew comwhi truth might get fire', ...,
       'michael j de la merc rachel abramsmaci said receiv takeov approach hudson bay new york time',
       'alex ansarynato russia hold parallel exercis balkan',
       'david swansonwhat keep f aliv'], dtype=object)

In [32]:
y

array([1, 0, 1, ..., 0, 1, 1])

### Converting the textual data into numerical data

In [34]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)



TfidfVectorizer()

In [35]:
X = vectorizer.transform(X)

In [36]:
print(X)

  (np.int32(0), np.int32(578))	0.2694167078545384
  (np.int32(0), np.int32(4211))	0.36253203231506576
  (np.int32(0), np.int32(5006))	0.24725958235728157
  (np.int32(0), np.int32(5969))	0.35488202138141456
  (np.int32(0), np.int32(6273))	0.2839932825877812
  (np.int32(0), np.int32(8022))	0.2313366174248873
  (np.int32(0), np.int32(12782))	0.24619727512767192
  (np.int32(0), np.int32(14555))	0.2917725968420029
  (np.int32(0), np.int32(15019))	0.4300622675963931
  (np.int32(0), np.int32(22724))	0.25523360180691607
  (np.int32(0), np.int32(26340))	0.2808837940159642
  (np.int32(1), np.int32(2622))	0.3562953366945267
  (np.int32(1), np.int32(3281))	0.18652439327549428
  (np.int32(1), np.int32(3859))	0.45980466668763476
  (np.int32(1), np.int32(4767))	0.23338756776626793
  (np.int32(1), np.int32(5916))	0.31810058109638056
  (np.int32(1), np.int32(8772))	0.5258635625386451
  (np.int32(1), np.int32(11313))	0.24166773097712638
  (np.int32(1), np.int32(27923))	0.36911845953845024
  (np.int32(2)

### Splitting the dataset to training and test data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=2)

# stratify keeps the same class distribution in train and test sets as in the original data.

### Train the model (Logistic Regression)

In [39]:
model = LogisticRegression()

In [40]:
model.fit(X_train, y_train)

LogisticRegression()

### Evaluation

In [41]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, y_train)

In [42]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.971875


In [43]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, y_test)

In [44]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9550480769230769


### Predictive Model

In [45]:
X_new = X_test[3]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[0]
The news is Real


In [47]:
print(y_test[3])

0
